### 1. Здійсніть імпорт необхідних пакетів.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns



### 2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv

In [2]:
# Завантаження тренувального набору
train_data = pd.read_csv("mod_04_hw_train_data.csv")

# Завантаження валідаційного набору
valid_data = pd.read_csv("mod_04_hw_valid_data.csv")

# Виведемо перші рядки для огляду даних
train_data.head(), valid_data.head()


(                 Name  Phone_Number  Experience  ... Cert Date_Of_Birth  Salary
 0  Jennifer Hernandez  120-602-1220         3.0  ...  Yes    25/08/1972   98000
 1      Timothy Walker  840-675-8650         5.0  ...  Yes    03/12/2013  135500
 2         David Duran  556-293-8643         5.0  ...  Yes    19/07/2002  123500
 3       Gloria Ortega  463-559-7474         3.0  ...   No    19/02/1970   85000
 4      Matthew Steele  968-091-7683         5.0  ...  Yes    20/02/1970  111500
 
 [5 rows x 9 columns],
               Name  Phone_Number  Experience  ... Cert Date_Of_Birth  Salary
 0   Alvaro Johnson  320-636-8883           7  ...   No    12/03/1978  109300
 1    Austin Powers  903-121-1691           2  ...  Yes    13/03/1992   84800
 2      Joshua Phil  673-972-2453           3  ...  Yes    19/02/1988   98900
 3  Mirinda Collins  310-364-6925           5  ...   No    20/03/1989  116500
 4   Mustapha Green  401-249-3912           3  ...  Yes    21/03/1979   75800
 
 [5 rows x 9 column

### 3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

In [3]:
# Перевіримо наявність пропусків у тренувальному та валідаційному наборах даних
missing_train = train_data.isnull().sum()
missing_valid = valid_data.isnull().sum()

In [4]:
missing_data_analysis = pd.DataFrame({
    "Train Missing Before": missing_train,
    "Train Missing After": train_data.isnull().sum(),
    "Valid Missing Before": missing_valid,
    "Valid Missing After": valid_data.isnull().sum()
})

# Виведемо результати аналізу пропусків у текстовому форматі
print("Аналіз пропусків у тренувальному наборі даних:")
print(missing_data_analysis[["Train Missing Before", "Train Missing After"]])

print("\nАналіз пропусків у валідаційному наборі даних:")
print(missing_data_analysis[["Valid Missing Before", "Valid Missing After"]])

Аналіз пропусків у тренувальному наборі даних:
               Train Missing Before  Train Missing After
Name                              0                    0
Phone_Number                      0                    0
Experience                        2                    2
Qualification                     1                    1
University                        0                    0
Role                              3                    3
Cert                              2                    2
Date_Of_Birth                     0                    0
Salary                            0                    0

Аналіз пропусків у валідаційному наборі даних:
               Valid Missing Before  Valid Missing After
Name                              0                    0
Phone_Number                      0                    0
Experience                        0                    0
Qualification                     0                    0
University                        0               

In [5]:
from datetime import datetime

# Видалимо стовпці 'Name' та 'Phone_Number'
train_data.drop(columns=['Name', 'Phone_Number'], inplace=True)
valid_data.drop(columns=['Name', 'Phone_Number'], inplace=True)

# Перетворимо 'Date_Of_Birth' у вік (кількість років)
current_year = datetime.now().year

train_data['Age'] = train_data['Date_Of_Birth'].apply(lambda x: current_year - int(x.split('/')[-1]))
valid_data['Age'] = valid_data['Date_Of_Birth'].apply(lambda x: current_year - int(x.split('/')[-1]))

# Видалимо стовпець 'Date_Of_Birth', оскільки ми вже отримали вік
train_data.drop(columns=['Date_Of_Birth'], inplace=True)
valid_data.drop(columns=['Date_Of_Birth'], inplace=True)

# Перевіримо, як тепер виглядають набори даних
train_data.head(), valid_data.head()


(   Experience Qualification University    Role Cert  Salary  Age
 0         3.0           Msc      Tier2     Mid  Yes   98000   52
 1         5.0           PhD      Tier2  Senior  Yes  135500   11
 2         5.0           Msc      Tier2  Senior  Yes  123500   22
 3         3.0           Bsc      Tier3     Mid   No   85000   54
 4         5.0           Bsc      Tier2  Senior  Yes  111500   54,
    Experience Qualification University    Role Cert  Salary  Age
 0           7           Bsc      Tier1  Senior   No  109300   46
 1           2           Msc      Tier1     Mid  Yes   84800   32
 2           3           Bsc      Tier3     Mid  Yes   98900   36
 3           5           Msc      Tier2  Senior   No  116500   35
 4           3           PhD      Tier1  Junior  Yes   75800   45)

In [6]:
# Заповнимо пропуски у тренувальному наборі
train_data.fillna({
    'Experience': train_data['Experience'].mean(),
    'Qualification': train_data['Qualification'].mode()[0],
    'Role': train_data['Role'].mode()[0],
    'Cert': train_data['Cert'].mode()[0]
}, inplace=True)

# Перевіримо, чи залишилися пропуски
missing_train_final = train_data.isnull().sum()
missing_train_final


Experience       0
Qualification    0
University       0
Role             0
Cert             0
Salary           0
Age              0
dtype: int64

### Крок 4: Масштабування числових ознак

In [7]:
# Ініціалізація трансформерів
scaler_standard = StandardScaler()
scaler_power = PowerTransformer()

# Вибір числових змінних для трансформації
numerical_features = ['Experience', 'Age']

# Масштабування за допомогою StandardScaler
train_data_standard = train_data.copy()
valid_data_standard = valid_data.copy()

train_data_standard[numerical_features] = scaler_standard.fit_transform(train_data[numerical_features])
valid_data_standard[numerical_features] = scaler_standard.transform(valid_data[numerical_features])

# Масштабування за допомогою PowerTransformer
train_data_power = train_data.copy()
valid_data_power = valid_data.copy()

train_data_power[numerical_features] = scaler_power.fit_transform(train_data[numerical_features])
valid_data_power[numerical_features] = scaler_power.transform(valid_data[numerical_features])

# Перевіримо результати трансформацій
train_data_standard.head(), train_data_power.head()


(   Experience Qualification University    Role Cert  Salary       Age
 0   -0.296683           Msc      Tier2     Mid  Yes   98000  1.346807
 1    1.047919           PhD      Tier2  Senior  Yes  135500 -2.079446
 2    1.047919           Msc      Tier2  Senior  Yes  123500 -1.160208
 3   -0.296683           Bsc      Tier3     Mid   No   85000  1.513942
 4    1.047919           Bsc      Tier2  Senior  Yes  111500  1.513942,
    Experience Qualification University    Role Cert  Salary       Age
 0   -0.367882           Msc      Tier2     Mid  Yes   98000  1.407452
 1    1.080821           PhD      Tier2  Senior  Yes  135500 -1.945406
 2    1.080821           Msc      Tier2  Senior  Yes  123500 -1.172305
 3   -0.367882           Bsc      Tier3     Mid   No   85000  1.597059
 4    1.080821           Bsc      Tier2  Senior  Yes  111500  1.597059)

In [8]:
# Ініціалізація OneHotEncoder для кодування категоріальних змінних
encoder = OneHotEncoder(sparse_output=False, drop='first')  # Drop='first' для уникнення мультиколінеарності
categorical_features = ['Qualification', 'University', 'Role', 'Cert']

# Кодування для StandardScaler трансформованих даних
encoded_train_standard = encoder.fit_transform(train_data_standard[categorical_features])
encoded_train_standard_df = pd.DataFrame(
    encoded_train_standard, 
    columns=encoder.get_feature_names_out(categorical_features)
)
train_data_standard_encoded = pd.concat(
    [train_data_standard.drop(columns=categorical_features), encoded_train_standard_df], axis=1
)

encoded_valid_standard = encoder.transform(valid_data_standard[categorical_features])
encoded_valid_standard_df = pd.DataFrame(
    encoded_valid_standard, 
    columns=encoder.get_feature_names_out(categorical_features)
)
valid_data_standard_encoded = pd.concat(
    [valid_data_standard.drop(columns=categorical_features), encoded_valid_standard_df], axis=1
)

# Кодування для PowerTransformer трансформованих даних
encoded_train_power = encoder.fit_transform(train_data_power[categorical_features])
encoded_train_power_df = pd.DataFrame(
    encoded_train_power, 
    columns=encoder.get_feature_names_out(categorical_features)
)
train_data_power_encoded = pd.concat(
    [train_data_power.drop(columns=categorical_features), encoded_train_power_df], axis=1
)

encoded_valid_power = encoder.transform(valid_data_power[categorical_features])
encoded_valid_power_df = pd.DataFrame(
    encoded_valid_power, 
    columns=encoder.get_feature_names_out(categorical_features)
)
valid_data_power_encoded = pd.concat(
    [valid_data_power.drop(columns=categorical_features), encoded_valid_power_df], axis=1
)

# Перевіримо, як виглядають дані після кодування
train_data_standard_encoded.head(), train_data_power_encoded.head()


(   Experience  Salary       Age  ...  Role_Mid  Role_Senior  Cert_Yes
 0   -0.296683   98000  1.346807  ...       1.0          0.0       1.0
 1    1.047919  135500 -2.079446  ...       0.0          1.0       1.0
 2    1.047919  123500 -1.160208  ...       0.0          1.0       1.0
 3   -0.296683   85000  1.513942  ...       1.0          0.0       0.0
 4    1.047919  111500  1.513942  ...       0.0          1.0       1.0
 
 [5 rows x 10 columns],
    Experience  Salary       Age  ...  Role_Mid  Role_Senior  Cert_Yes
 0   -0.367882   98000  1.407452  ...       1.0          0.0       1.0
 1    1.080821  135500 -1.945406  ...       0.0          1.0       1.0
 2    1.080821  123500 -1.172305  ...       0.0          1.0       1.0
 3   -0.367882   85000  1.597059  ...       1.0          0.0       0.0
 4    1.080821  111500  1.597059  ...       0.0          1.0       1.0
 
 [5 rows x 10 columns])

### Крок 5: Побудова моделі за допомогою KNeighborsRegressor

In [9]:
# Розділення даних на ознаки (features) та цільову змінну (target)
X_train_standard = train_data_standard_encoded.drop(columns=['Salary'])
y_train_standard = train_data_standard_encoded['Salary']

X_valid_standard = valid_data_standard_encoded.drop(columns=['Salary'])
y_valid_standard = valid_data_standard_encoded['Salary']

# Побудова моделі KNeighborsRegressor для StandardScaler даних
knn_model_standard = KNeighborsRegressor(n_neighbors=5)
knn_model_standard.fit(X_train_standard, y_train_standard)

# Прогноз для валідаційного набору
y_pred_standard = knn_model_standard.predict(X_valid_standard)




### Крок 6: Оцінка точності моделі (MAPE)

In [10]:
from sklearn.metrics import mean_absolute_percentage_error
# Обчислення MAPE
mape_standard = mean_absolute_percentage_error(y_valid_standard, y_pred_standard)

# Аналогічно для PowerTransformer
X_train_power = train_data_power_encoded.drop(columns=['Salary'])
y_train_power = train_data_power_encoded['Salary']

X_valid_power = valid_data_power_encoded.drop(columns=['Salary'])
y_valid_power = valid_data_power_encoded['Salary']

knn_model_power = KNeighborsRegressor(n_neighbors=5)
knn_model_power.fit(X_train_power, y_train_power)

y_pred_power = knn_model_power.predict(X_valid_power)
mape_power = mean_absolute_percentage_error(y_valid_power, y_pred_power)

# Виведемо результати
print(f"Validation MAPE (StandardScaler): {mape_standard:.2%}")
print(f"Validation MAPE (PowerTransformer): {mape_power:.2%}")

Validation MAPE (StandardScaler): 11.11%
Validation MAPE (PowerTransformer): 11.11%


### Крок 7: Розрахунок додаткових метрик точності (MSE, R2)

In [11]:
# Розрахуємо метрики точності регресійної моделі для StandardScaler

# Mean Squared Error (MSE) для StandardScaler
mse_standard = mean_squared_error(y_valid_standard, y_pred_standard)

# R-squared (R2) для StandardScaler
r2_standard = r2_score(y_valid_standard, y_pred_standard)

# Розрахуємо метрики для PowerTransformer
mse_power = mean_squared_error(y_valid_power, y_pred_power)
r2_power = r2_score(y_valid_power, y_pred_power)

# Виведемо результати
print(f"StandardScaler Results:")
print(f" - Validation MAPE: {mape_standard:.2%}")
print(f" - Validation MSE: {mse_standard:.2f}")
print(f" - Validation R2: {r2_standard:.2f}")

print(f"\nPowerTransformer Results:")
print(f" - Validation MAPE: {mape_power:.2%}")
print(f" - Validation MSE: {mse_power:.2f}")
print(f" - Validation R2: {r2_power:.2f}")


StandardScaler Results:
 - Validation MAPE: 11.11%
 - Validation MSE: 140841428.57
 - Validation R2: 0.44

PowerTransformer Results:
 - Validation MAPE: 11.11%
 - Validation MSE: 140841428.57
 - Validation R2: 0.44


### Висновок:

1. MAPE (Mean Absolute Percentage Error): Обидва методи масштабування, StandardScaler і PowerTransformer, продемонстрували однаковий результат MAPE на рівні 11.11%. Це свідчить про те, що обраний спосіб трансформації числових змінних суттєво не вплинув на якість моделі.

2. MSE (Mean Squared Error): Значення середньоквадратичної помилки (MSE) для обох методів також однакове та становить 140841428.57. Це вказує на те, що модель має однакову помилку в прогнозах незалежно від використаного методу масштабування.

3. R2 (Коефіцієнт детермінації): Значення R2 дорівнює 0.44, що свідчить про те, що модель пояснює лише 44% варіації цільової змінної. Це середній рівень пояснювальної здатності.

4. Оцінка масштабування: Незважаючи на використання двох методів (StandardScaler і PowerTransformer), вони не вплинули на результативність моделі. Це може вказувати на те, що числові змінні вже мали оптимальний розподіл для методу KNeighborsRegressor.

5. Рекомендації: Для покращення моделі можна:

    - Спробувати налаштувати параметр n_neighbors у KNeighborsRegressor.
    - Дослідити важливість кожної ознаки для виявлення потенційно "шумових" змінних.
    - Використати інші алгоритми регресії, такі як RandomForestRegressor або GradientBoostingRegressor, які можуть забезпечити кращу пояснювальну здатність.
    - Виконати додаткову інженерію ознак, зокрема створити нові змінні або перетворити існуючі.